In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import numpy as np
from utilities import remove_cols

samples = np.genfromtxt('data/samples.csv', delimiter=",")

In [7]:


# inputs and outputs
X = remove_cols(np.genfromtxt('data/orth.csv', delimiter=","))
Y = remove_cols(np.genfromtxt('data/phon.csv', delimiter=","))



From `tune_1` we saw that a configuration with learning rate of .25, batch size of 64, and number of epochs of 100 produced good train (.978) and test (.965) accuracy despite a short training time. Let's search around those parameters to see if we can push a little closer to 1.

In [134]:

hidden = 100
seed = 387
import time

def learner(seed, learning_rate, hidden):
    np.random.seed(seed)

    optimizer = Adam(learning_rate=learning_rate) # was originally .1

    model = Sequential()
    model.add(Dense(hidden, input_shape=(X.shape[1],)))
    model.add(Dense(Y.shape[1], activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 'binary_crossentropy', 'mse'])
    
    return model

with open('outputs/tune_2.csv', 'w') as f:
    f.write("{},{},{},{},{},{},{},{},{},{}\n".format("learning_rate",
                                             "batch_size",
                                             "epochs",
                                             "loss_train",
                                             "accuracy_train",
                                             "mse_train",
                                             "loss_test",
                                             "accuracy_test",
                                             "mse_test",
                                             "time"))
    for learning_rate in [.19, .20, .21, .22, .23, .24, .25]: 
        for batch_size in [32, 48, 64, 80, 96]:
            for epochs in [100, 125, 150, 175, 200, 225]:
                model = learner(seed, learning_rate, hidden)
                
                start_time = time.time()


                model.fit(X[sample], Y[sample], epochs=epochs, batch_size=batch_size, verbose=False)

                end_time = time.time()
                runtime = end_time - start_time

                loss_train, accuracy_train, __, mse_train = model.evaluate(X[sample], Y[sample], verbose=0) # loss is duplicated so the 2nd value is set to __
                loss_test, accuracy_test, __, mse_test = model.evaluate(X[~sample], Y[~sample], verbose=0) # loss is duplicated so the 2nd value is set to __

                f.write("{},{},{},{},{},{},{},{},{},{}\n".format(learning_rate,
                                                batch_size,
                                                epochs,
                                                loss_train,
                                                accuracy_train,
                                                mse_train,
                                                loss_test,
                                                accuracy_test,
                                                mse_test,
                                                runtime))
f.close()